<a href="https://colab.research.google.com/github/Chiamakac/TRAININGS/blob/main/TRAININGS/Alignment/Projection_Work/TAG_ENG/Updating_the_Mapping_Dictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


---

#### **Using the `mapping_dict` to transfer NER tags from Eng to Igbo**

In [1]:
# from collections import Counter
import pickle

Read the tagged English corpus

In [2]:
!wget -c https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/Projection_Work/TAG_ENG/spaCy%20en_tags%20corrected%20FINAL%20USED.txt

--2022-10-13 12:00:08--  https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/Projection_Work/TAG_ENG/spaCy%20en_tags%20corrected%20FINAL%20USED.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 288470 (282K) [text/plain]
Saving to: ‘spaCy en_tags corrected FINAL USED.txt’

spaCy en_tags corre 100%[===================>] 281.71K  --.-KB/s    in 0.02s   

2022-10-13 12:00:09 (11.1 MB/s) - ‘spaCy en_tags corrected FINAL USED.txt’ saved [288470/288470]



In [2]:
tagged_sents = open('/content/spaCy en_tags corrected FINAL USED.txt').readlines()
tagged_sents = [tagged.strip() for tagged in tagged_sents]

In [3]:
tagged_entities = {}
for line in tagged_sents:
  lnum, line = line.strip().split(', ', 1)
  tagged_entities[int(lnum)] = line.split('|')

Read the English-Igbo BiText

In [ ]:
!wget -c https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/Projection_Work/TAG_ENG/projection_en_bitext.txt

--2022-10-13 10:18:14--  https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/Projection_Work/TAG_ENG/projection_en_bitext.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



Extract the Igbo Text only

In [4]:
ig_text = open('projection_ig_text.txt').readlines()

**Loading the mapping dictionary**

In [19]:
# count the remaining entities not found
def count_not_found():
  return sum([values['ig'].startswith('Not found') for entity, values in mapping_dict.items()])

#save the current version of mapping dictionary
def save_dict(m_dict):
  with open('mapping_dict.pkl', 'wb') as handle:
    pickle.dump(m_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
  print("Mapping dictionary successfully saved to 'mapping_dict.pkl'")

#load the latest version of the mapping dictionary
def load_dict(fname_pkl):
  with open(fname_pkl, 'rb') as handle:
    return pickle.load(handle)

In [20]:
load_mdict = input('Do you want to load mapping dictionary? [Y/N]')
if load_mdict.lower() == 'y':
  mapping_dict = load_dict('mapping_dict.pkl')
  print("Mapping dictionary successfully loaded from 'mapping_dict.pkl'")

Do you want to load mapping dictionary? [Y/N]y
Mapping dictionary successfully loaded from 'mapping_dict.pkl'


**Core Transfer Process** (Work in progress...)

In [30]:
ig_text = open('projection_ig_text.txt').readlines()

In [31]:
with open('ig_transferred_ner_tags.txt', 'w', encoding='utf8') as ig_tagged_file:
  try:
    for lineNum in tagged_entities:
      entities_tags, ig_sent = tagged_entities[lineNum], ig_text[lineNum]
      for entity_tag in entities_tags:
        entity, tag = entity_tag.split(', ') #check this later
        ig_entity = mapping_dict[entity.lower()]['ig'].strip()
        tag = mapping_dict[entity.lower()]['tag']
        ent_start_index = ig_sent.find(ig_entity)
        # print(ig_entity, ent_start_index)
        if ent_start_index > -1:
          left_context = ig_sent[:ent_start_index]
          ig_tagged_file.write("\n".join([f"{tkn.strip()} O" for tkn in left_context.split()]))
          if ' ' not in ig_entity:
            ig_tagged_file.write(f"\n{ig_entity} B-{next(iter(tag))}")
          else:
            entity_tokens = ig_entity.split()
            entity_tok_tags = [f"{tkn.strip()} B-{next(iter(tag))}" if i==0 else 
                              f"{tkn.strip()} I-{next(iter(tag))}"
                              for i, tkn in enumerate(entity_tokens)]
            formated_tok_tags = '\n'.join(entity_tok_tags)
            ig_tagged_file.write(f"\n{formated_tok_tags}")
          right_context = ig_sent[ent_start_index+len(ig_entity):]
          ig_tagged_file.write(f"\n{'-'*3}\n")
          ig_sent = right_context
        else:
          print(f"Not found-> Line {lineNum} en: '{entity}' ig: '{ig_entity}'")
  except ValueError:
      print(f"ValueError: Line {lineNum}\n-entities_tags: '{entities_tags}'\n-ig_sent: '{ig_sent}'")

Not found-> Line 76 en: 'South African' ig: 'ndi ndida afrịka'
ValueError: Line 103
-entities_tags: '['February 20, 2020, DATE']'
-ig_sent: 'otu ọ dị ịnụ maka okwu a abụrụla nke ebugharịrị rue nụbọchị nke iri abụọ nọnwa februwarị nafọ 2020 nọmụmatụ ikpe ahụ 
'


**Working with Mapping Dictionary**

You can `[V]iew`, `[A]dd`, `[R]emove` or `[U]pdate` entries in the mapping dictionary

In [22]:
correct = input('[V]iew, [A]dd, [R]emove or [U]pdate entry in the mapping_dict? [C]ancel ')
while correct.lower() != 'c':
# --------- view
  if correct.lower()=='v':
    view_entity = input('Enter entity to view ')
    if view_entity in mapping_dict:
      print(f"'{view_entity}' contains '{str(mapping_dict[view_entity])}'")
    else:
      print(f"'{view_entity}' not found!")
# --------- add
  elif correct.lower()=='a':
    add_entity = input('Enter entity to add ')
    if add_entity not in mapping_dict:
      new_ig_entity = input('Enter ig entity ')
      new_tag = input('Enter tag ')
      mapping_dict[add_entity] = {'ig':new_ig_entity, 'tag':{new_tag}}
      print(f"'{str(mapping_dict[add_entity])}' added successfully")
    else:
      print(f"'{add_entity}' already exists!")
# --------- remove
  elif correct.lower()=='r':
    remove_entity = input('Enter entity to remove ')
    if remove_entity in mapping_dict:
      removed = mapping_dict.pop(remove_entity)
      print(f"'{str(removed)}' successfully removed!")
    else:
      print(f"'{remove_entity}' not found!")
# --------- update
  elif correct.lower()=='u':
    update_entity = input('Enter entity to update ')
    if update_entity in mapping_dict:
      update_ig_entity = input(f"Update ig entity '{mapping_dict[update_entity]['ig']}'? [Y/N] ")
      if update_ig_entity.lower() == 'y':
        old_ig_entity = mapping_dict[update_entity]['ig']
        new_ig_entity = input('Enter new ig entity: ')
        mapping_dict[update_entity]['ig'] = new_ig_entity if new_ig_entity else old_ig_entity
        print(f" Changed ig entity: '{old_ig_entity}'==>'{new_ig_entity}'")
      update_tag = input(f"Update tag '{mapping_dict[update_entity]['tag']}'? [Y/N] ")
      if update_tag.lower() == 'y':
        old_tag = mapping_dict[update_entity]['tag']
        new_tag = input('Enter new tag: ')
        mapping_dict[update_entity]['tag'] = {new_tag} if new_tag else old_tag
        print(f"Changed ig entity: '{old_tag}'==>'{new_tag}'")
      print(f"'{str(mapping_dict[update_entity])}' updated successfully")
    else:
      print(f"'{update_entity}' not found!")
  else:
    print(f"Invalid input '{correct}'!")
  correct = input(f"{'-'*10}\n[V]iew, [A]dd, [R]emove or [U]pdate entry in the mapping_dict? [C]ancel ")
print("Thanks...")

[V]iew, [A]dd, [R]emove or [U]pdate entry in the mapping_dict? [C]ancel u
Enter entity to update nigerian
Update ig entity 'nde iri atọ naịjirịa'? [Y/N] y
Enter new ig entity: ndi naịjirịa
 Changed ig entity: 'nde iri atọ naịjirịa'==>'ndi naịjirịa'
Update tag '{'NORP'}'? [Y/N] y
Enter new tag: NORP
Changed ig entity: '{'NORP'}'==>'NORP'
'{'ig': 'ndi naịjirịa', 'tag': {'NORP'}}' updated successfully


KeyboardInterrupt: ignored

**Updating the mapping dictionary**

In [23]:
update_mdict = input('Do you want to update mapping dictionary? [Y/N]')
if update_mdict.lower() == 'y':
  save_dict(mapping_dict)

Do you want to update mapping dictionary? [Y/N]Y
Mapping dictionary successfully saved to 'mapping_dict.pkl'


In [24]:
mapping_dict

{'nigeria': {'ig': 'naịjirịa', 'tag': {'LOC'}},
 'omoyele sowore': {'ig': 'omoyele sowore', 'tag': {'PER'}},
 'more than 120 days': {'ig': 'karịrị abalị otu narị na iri abụọ',
  'tag': {'DATE'}},
 'sowore': {'ig': 'sowore', 'tag': {'PER'}},
 'the middle of the day': {'ig': "n'etiti ehihie", 'tag': {'DATE'}},
 'every year': {'ig': 'kwa afọ', 'tag': {'DATE'}},
 'the end of every year': {'ig': 'ngwụsị afọ ọbụla', 'tag': {'DATE'}},
 'today': {'ig': 'taa', 'tag': {'DATE'}},
 '1984': {'ig': '1984', 'tag': {'DATE'}},
 'monaco': {'ig': 'monaco', 'tag': {'LOC'}},
 'first': {'ig': 'mbụ', 'tag': {'ORDINAL'}},
 '30 million': {'ig': 'nde iri atọ', 'tag': {'CARDINAL'}},
 'nigerian': {'ig': 'ndi naịjirịa', 'tag': {'NORP'}},
 'efcc': {'ig': 'efcc', 'tag': {'ORG'}},
 'more than $ 7 billion': {'ig': "n'ihe karịrị ijeri dọla asaa",
  'tag': {'MONEY'}},
 'wilson uwujaren': {'ig': 'wilson uwujaren', 'tag': {'PER'}},
 'mohammed idris': {'ig': 'mohammed idris', 'tag': {'PER'}},
 'december 2019': {'ig': "dise